<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Иморт-бибилиотек" data-toc-modified-id="Иморт-бибилиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Иморт бибилиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Обработка-текста-и-деление" data-toc-modified-id="Обработка-текста-и-деление-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Обработка текста и деление</a></span><ul class="toc-item"><li><span><a href="#Деление-на-тренировочную-и-тестовую-выборки" data-toc-modified-id="Деление-на-тренировочную-и-тестовую-выборки-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Деление на тренировочную и тестовую выборки</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Random-forest" data-toc-modified-id="Random-forest-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Random forest</a></span></li><li><span><a href="#Logistic-regression" data-toc-modified-id="Logistic-regression-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Logistic regression</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-2.0.4"><span class="toc-item-num">2.0.4&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка
### Иморт бибилиотек

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')


### Загрузка данных

In [15]:
try:
    df = pd.read_csv('./datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Текст на английском, поэтому будем исопльзовать обычный BERT.  

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Типы данных в порядке. В датасете около 160 тыс. фраз.

In [17]:
df.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Целевой столбец - булевый, что нам и требовалось.
Доля токсичных комментариев - чуть больше 10%  
Не понятно, что за столбец unnamed, похоже на дублирование индекса, проверим.

Все верно, это дубликат, удалим его.

In [18]:
df = df[['text','toxic']]
df.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Уменьшим датафрейм до 30000. При необходимости увеличим позже.

In [19]:
df = df.sample(80000,random_state=12345)
df['toxic'].mean()

0.1011375

### Обработка текста и деление

#### Деление на тренировочную и тестовую выборки

In [20]:
train = df.sample(frac=0.8, random_state=42)
test = df.drop(train.index)


In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\atropinskiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
#stopwords = set(nltk_stopwords.words('english'))
stopwords = 'english'
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [28]:
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

train['formatted_text'] = train['text'].apply(cleaning)
test['formatted_text'] = test['text'].apply(cleaning)

y_train = train['toxic']
y_test = test['toxic']

train.head()

,text,toxic,formatted_text
156947,Check Out Mentone Grammar School's talk page\n...,0,check out mentone grammar schools talk page i...
95179,I agree with and that reference to the longe...,0,i agree with and that reference to the longe...
54035,Julie Greene <3 \n\nTeddy Barrett.\nIs Trouble...,0,julie greene teddy barrett is trouble and i...
75962,"""\nI agree. A simple map of the area would suf...",0,i agree a simple map of the area would suffice...
63634,Contested deletion \n\nThis article should not...,0,contested deletion this article should not b...


In [29]:
tfidf_train = count_tf_idf.fit_transform(train['formatted_text'])
tfidf_test = count_tf_idf.transform(test['formatted_text'])

## Обучение

#### Random forest

In [30]:
model_rf = RandomForestClassifier(random_state=12345)

params_rf = {
    'n_estimators': [13],
    'max_depth':[5],
    'max_features' : [10000]
}

grid = GridSearchCV(model_rf, param_grid=params_rf, scoring='f1', cv=3, verbose=True, n_jobs=-1)

best_grid = grid.fit(tfidf_train, y_train)
print('Best parameters is:', grid.best_params_)
print('Best score for RF is:', grid.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters is: {'max_depth': 5, 'max_features': 10000, 'n_estimators': 13}
Best score for RF is: 0.23370604548252846


#### Logistic regression

In [31]:
model_lr = LogisticRegression()

parameters_lr = {
    'penalty' : ['l1','l2'], 
    'solver'  : ['lbfgs', 'liblinear'],
}

grid_lr = GridSearchCV(model_lr,                    
                   param_grid = parameters_lr,  
                   scoring='f1',
                   cv=3)
grid_lr.fit(tfidf_train, y_train)

print('Best parameters is:', grid_lr.best_params_)
print('Best score for RF is:', grid_lr.best_score_)

Best parameters is: {'penalty': 'l1', 'solver': 'liblinear'}
Best score for RF is: 0.7344290297341093


#### Catboost

In [32]:
# Кэтбуст умеет самостоятельно векторизировать текст, поэтому оставим ему только обработанный текст в качестве признаков.
X_train = train.drop(['toxic', 'text'],axis=1)
X_test = test.drop(['toxic', 'text'],axis=1)

In [33]:
catboost = CatBoostClassifier(logging_level = 'Silent',
                              text_features=['formatted_text'])

parameters_cb = {'learning_rate': [0.1], # Оставил лучшие параметры для экономии времени
                'l2_leaf_reg': [1],
                'iterations' : [500]}

grid_cb  = GridSearchCV(estimator=catboost, 
                     param_grid = parameters_cb, 
                     cv = 2, 
                     n_jobs=-1,
                     scoring='f1')

grid_cb.fit(X_train, y_train)

print ('Лучший результат:', grid_cb.best_score_)
print ('Лучшие параметры:', grid_cb.best_params_)

Лучший результат: 0.7565341719897716
Лучшие параметры: {'iterations': 500, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


#### Тестирование

In [34]:

test_pred = grid_cb.predict(X_test)
print ('Best f1 score for CB is:',f1_score(y_test, test_pred))
test_pred = grid_lr.predict(tfidf_test)
print ('Best f1 score for LR is:',f1_score(y_test, test_pred))

Best f1 score for CB is: 0.7710184552289817
Best f1 score for LR is: 0.7401978746793697


## Выводы

1. В данном проекте мы преобразовали текст, токенизировали его, удалили стоп-слов, что позволило нам построить несколько моделей.
2. Изначально мы уменьшили количество данных до 80000, чтобы сэкономить ресурсы
3. Лучше всех отработал Catboost, стоит отметить, что для построения модели с определенными гиперпараметрами ему хватило и 30тыс. данных, что является отличным результатом. И это позволило ему превысить порог 75% в f1, что и требовалось в проекте.
4. Также неплохо отработала Логическая регрессия, но с учетом кроссвалидации на тестовой выборке она не дала нужного результата
5. Модель RF отработала хуже. всех.

В результате предлагаю использовать модель catboost, которая показала лучший результат по точности, также у нас остается резерв на улучшение качества модели, не урезая до 80000 начальный датафрейм.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны